# Sessão Spark e Configurações Iniciais

In [80]:
from pyspark.sql import SparkSession

In [81]:
spark

In [82]:
# Inicializa a sessão Spark
spark = SparkSession.builder.appName("ProcessRawToTrusted").getOrCreate()

# Configura o acesso ao MinIO
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", "datalake")
hadoop_conf.set("fs.s3a.secret.key", "datalake")
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [165]:
# Defina o caminho de entrada e saída com base na data
import pytz
from datetime import datetime

fuso_horario_brasilia = pytz.timezone('America/Sao_Paulo')
brasilia_time = datetime.now(fuso_horario_brasilia)
today = brasilia_time.strftime('%Y-%m-%d')

raw_path = f"s3a://raw/busposition/{today}"
trusted_path = f"s3a://trusted/busposition/{today}"

routes_trusted_path = f"s3a://trusted/routes/{today}/"
positions_trusted_path = f"s3a://trusted/positions/{today}/"

2024-10-13


# ETL da Camanda Raw para Trusted

In [172]:

from pyspark.sql.functions import explode, col
# Exibe a lista de arquivos
files_df = spark.read.format("binaryFile").load(raw_path)
# Itera sobre os arquivos
for row in files_df.collect():
    raw_file_path = row["path"]
    raw_file_date = raw_file_path.split('_')[-1].split('.')[0]
    df_raw = spark.read.json(raw_file_path)
    
    # Explodir o array "l" para acessar as informações de cada linha de ônibus
    df_lines = df_raw.select(explode(col("l")).alias("linha"))

    # DataFrame com as informações de "c" até "qv"
    df_bus_lines = df_lines.select(
        col("linha.cl").alias("codigo_trajeto"),
        col("linha.sl").alias("sentido"),
        col("linha.c").alias("letreiro"),
        col("linha.lt0").alias("terminal_primario"),
        col("linha.lt1").alias("terminal_secundario"),
        col("linha.qv").alias("qnt_veiculos")
    )

    # Explodir o array "vs" para acessar as informações de cada veículo dentro da linha de ônibus
    df_vehicles = df_lines.select(
        col("linha.cl").alias("codigo_trajeto"),
        col("linha.sl").alias("sentido"),
        explode(col("linha.vs")).alias("vehicle")
    )

    # DataFrame com as informações de "p" até "px"
    df_vehicles_position = df_vehicles.select(
        col("codigo_trajeto"),
        col("sentido"),
        col("vehicle.p").alias("prefixo_veiculo"),
        col("vehicle.py").alias("latitude"),
        col("vehicle.px").alias("longitude")
    )
    
    # Salva os DataFrames no caminho definido
    #df_bus_lines.write.mode("overwrite").json(routes_trusted_path + 'routes_' + raw_file_date)
    #df_vehicles_position.write.mode("overwrite").json(positions_trusted_path + 'positions_' + raw_file_date)


In [173]:
df_bus_lines.show()

+--------------+-------+--------+--------------------+--------------------+------------+
|codigo_trajeto|sentido|letreiro|   terminal_primario| terminal_secundario|qnt_veiculos|
+--------------+-------+--------+--------------------+--------------------+------------+
|           780|      1| 736G-10|       SHOP. MORUMBI|            JD. INGÁ|           2|
|           846|      1| 1702-10|               TIETÊ|          JOVA RURAL|           6|
|         33893|      2| 5123-10|     HOSP. SÃO PAULO|          JD. MIRIAM|           3|
|         35230|      2| 709M-10|     TERM. PINHEIROS|    TERM. STO. AMARO|           2|
|          1986|      1| 6076-41|    TERM. STO. AMARO|     JD. SATÉLITE II|           1|
|         32877|      2| 675R-10|     METRÔ JABAQUARA|              GRAJAÚ|           3|
|         33432|      2| 172N-10|         METRÔ BELÉM|  SHOP. CENTER NORTE|           6|
|         33157|      2| 273J-10|         METRÔ PENHA|          JD. ROMANO|           8|
|          1887|     

In [174]:
df_vehicles_position.show()

+--------------+-------+---------------+-------------------+-------------------+
|codigo_trajeto|sentido|prefixo_veiculo|           latitude|          longitude|
+--------------+-------+---------------+-------------------+-------------------+
|           780|      1|          86177|        -23.6190115|        -46.6997825|
|           780|      1|          86366|         -23.618506|         -46.699464|
|           846|      1|          25638|         -23.442846|          -46.57655|
|           846|      1|          25633|        -23.4789755|        -46.5991595|
|           846|      1|          25635|-23.504054125000003|-46.624669499999996|
|           846|      1|          25211|-23.494891000000003|        -46.6106765|
|           846|      1|          25650|      -23.466649875|-46.583931375000006|
|           846|      1|          25220|        -23.4454635|-46.577521000000004|
|         33893|      2|          68167|-23.678414500000002|       -46.63006525|
|         33893|      2|    